In [1]:
import numpy as np
from packages.Utility.CNN.Callbacks import *
from packages.Utility.DatasetManager import DatasetReader
from packages.Utility.Gabor import GaborFilterBanks, GaborParams
from packages.Model.CFNet import CFNet1_1

Using TensorFlow backend.


# Inisialisasi / Persiapan

In [2]:
fold_index = 0
dataset_dir = 'dataset/bin/CF.32.B.30K'
trained_dir = 'trained/CF.32.B.30K CF1.1.2'

# Memuat Dataset

In [3]:
reader = DatasetReader(dataset_dir)

X_train, Y_train_onehot, Y_train_single = reader.getTrainData(fold_index)
X_test, Y_test_onehot, Y_test_single = reader.getTestData(fold_index)

# Inisiasi Callbacks

In [4]:
val_model = ValidateModel(X_train, Y_train_onehot)
save_model = SaveModel(val_model, base_dir = trained_dir, fold = fold_index + 1, min_acc = 0.1)
stop_train = StopTraining(val_model)
history_save = HistorySaver(val_model, base_dir = trained_dir, fold = fold_index + 1)

# Parameter Gabor

In [5]:
class ChannelizeGaborParams(GaborParams):
    
    def getParams(self, shape, i, j):
        multiplier = 360 / shape[3]
        degree = multiplier * i
        
        i += 1
        j += 1
        
        lambd = (i * j) / shape[2]
        theta = degree * np.pi / 180
        psi = (i + j) / shape[2]
        sigma = (i + j)
        gamma = (i + j) / shape[2]
        
        return lambd, theta, psi, sigma, gamma

# Mendefinisikan Kernel Gabor

In [6]:
kernel1 = GaborFilterBanks(ChannelizeGaborParams())
kernel2 = GaborFilterBanks(ChannelizeGaborParams())
kernel3 = GaborFilterBanks(ChannelizeGaborParams())
kernel4 = GaborFilterBanks(ChannelizeGaborParams())
kernel5 = GaborFilterBanks(ChannelizeGaborParams())
kernel6 = GaborFilterBanks(ChannelizeGaborParams())

# Mendefinisikan Model

In [7]:
ars = CFNet1_1(kernels = {
    1: kernel1,
    2: kernel2,
    3: kernel3,
    4: kernel4,
    5: kernel5,
    6: kernel6
}, loss = 'categorical_crossentropy', optimizer = 'adadelta')
model = ars.model

In [ ]:
ars.save(f'{trained_dir}/Fold {fold_index + 1}/init.h5')
model.summary()

In [8]:
model.load_weights(f'trained/CF.32.B.30K CF1.1.2/Fold 1/acc 0.9097 - epoch 297.h5')

# Pelatihan

In [10]:
ars.compile()
history = model.fit(X_train, Y_train_onehot, epochs = 500, verbose = 2, batch_size = 128, callbacks=[val_model, save_model, history_save, stop_train], initial_epoch = 400)

Epoch 401/500
 - 100s - loss: 0.0957 - acc: 0.9704
New best accuracy : 0.9942222222222222
Model saved to disk
Passes : 0/50
Epoch 402/500
 - 99s - loss: 0.0752 - acc: 0.9802
Accuracy : 0.9786222222222222
Model saved to disk
Passes : 1/50
Epoch 403/500
 - 99s - loss: 0.0698 - acc: 0.9828
Accuracy : 0.9834666666666667
Model saved to disk
Passes : 2/50
Epoch 404/500
 - 99s - loss: 0.0672 - acc: 0.9842
Accuracy : 0.9816444444444444
Model saved to disk
Passes : 3/50
Epoch 405/500
 - 99s - loss: 0.0676 - acc: 0.9838
Accuracy : 0.9876
Model saved to disk
Passes : 4/50
Epoch 406/500
 - 99s - loss: 0.0658 - acc: 0.9843
Accuracy : 0.9907111111111111
Model saved to disk
Passes : 5/50
Epoch 407/500
 - 99s - loss: 0.0654 - acc: 0.9847
Accuracy : 0.9704444444444444
Model saved to disk
Passes : 6/50
Epoch 408/500
 - 100s - loss: 0.0628 - acc: 0.9858
Accuracy : 0.9787111111111111
Model saved to disk
Passes : 7/50
Epoch 409/500
 - 100s - loss: 0.0613 - acc: 0.9866
Accuracy : 0.9736
Model saved to disk
